In [1]:
# import necessary libraries
import pandas as pd
import re
from airportsdata import load
from actions.done.CreateZFWMessageAction import CreateZFWMessageAction
from actions.done.UpdateEstimatesAction import UpdateEstimatesAction
import json

def load_config(path):
    with open(path, 'r') as file:
        return json.load(file)

config_path = '../pandas_config.json'
config = load_config(config_path)
parquet_config = config['parquet']

# Setting up the engine as a global constant
ENGINE = parquet_config['engine']
COMPRESSION = parquet_config['compression']
FILE_PATH = "../data"

In [30]:
df = pd.read_parquet(f"{FILE_PATH}/data_parquet/processed_data_combined.parquet", engine=ENGINE)

In [31]:
df = df[df["action_name"].isin(["CreateZFWMessageAction", "UpdateEstimatesAction"])]

In [39]:
flight_numbers = df['flight_number'].unique()
flight_numbers = [flight_number for flight_number in flight_numbers if flight_number != None]

In [42]:
flight_numbers

['ba653ba04189284d',
 '1eb7ca10727068bb',
 'b207bbfe3e4f9f10',
 '2efde9a356114794',
 '7113033194c3461f',
 'fec6ed7349f101ed',
 '516d352921ada3d',
 '59e12b7a42fce5dd',
 '9b9dcc26173558c9',
 '43f94f6c2cd482d2',
 '8eea40f447ae37f8',
 '2e126d055ef28d45',
 'd07cb75518dd7a4e',
 'f30d76da49920a6b',
 'a7657cdff2d4dd8f',
 '7575bbfbff8d8add',
 '215b6e59c204d81a',
 'acc4be119fe087bf',
 'ed55e4e8f1b53cd7',
 'ecad4a8a8f909343',
 'b04181fdf29965f9',
 'ec16a4d0bd73f945',
 '87032d6a4e7f8892',
 '5851d5f7e3c9b23',
 'bcd53a0c5a38f3d7',
 'd6f0120f3b93186e',
 '2507fd7485778b8c',
 '499137b6233829cb',
 'd3a649c8703dcc26',
 '72ff9b04e133cf8',
 '896cbf13f526f024',
 'f97e2b859b0b2957',
 'dcde8d585bd18f7e',
 'd784fc23c96365f',
 '5d2ee1f150dc291a',
 '917b67da42e2b2cc',
 '1f3ad5d482bc5602',
 'deb0043f128c2ea5',
 'bbed866d4468ba5a',
 '81c699641a66184f',
 '593f6dcdea5cdfbe',
 '41ceadbcc2a8c94e',
 '11a2268aa2e27b13',
 '3662c3d9ff3864af',
 '1e80ca0ad91bf6a2',
 'c596adc0d999f261',
 'ff61a942adcad96e',
 'adc5572f3b651d6

In [47]:
estimated_zfws = {}
actual_zfws = {}
for flight in flight_numbers[:100]:
    subset_df = df[df['flight_number'] == flight]
    subset_df.sort_index(inplace=True)
    actions = subset_df['action_name'].values
    entriy_details = subset_df["entry_details"].values
    temp_estimated_zfws = []
    temp_actual_zfws = []
    for entry, action in zip(entriy_details, actions):
        if action == "CreateZFWMessageAction":
            estimated_zfw, actual_zfw = CreateZFWMessageAction(entry)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
        if action == "UpdateEstimatesAction":
            estimated_zfw, actual_zfw = UpdateEstimatesAction(entry)
            if estimated_zfw != None:
                temp_estimated_zfws.append(estimated_zfw)
            if actual_zfw != None:
                temp_actual_zfws.append(actual_zfw)
    estimated_zfws[flight] = temp_estimated_zfws
    actual_zfws[flight] = temp_actual_zfws
    #estimated_zfws.append(temp_estimated_zfws)
    #actual_zfws.append(temp_actual_zfws)

In [48]:
actual_zfws

{'ba653ba04189284d': [127386, 127386],
 '1eb7ca10727068bb': [58225, 58225],
 'b207bbfe3e4f9f10': [54608],
 '2efde9a356114794': [54608],
 '7113033194c3461f': [54608],
 'fec6ed7349f101ed': [54608],
 '516d352921ada3d': [54608],
 '59e12b7a42fce5dd': [54608],
 '9b9dcc26173558c9': [54608],
 '43f94f6c2cd482d2': [54608, 54608],
 '8eea40f447ae37f8': [54608, 54608],
 '2e126d055ef28d45': [52957],
 'd07cb75518dd7a4e': [52957, 52957],
 'f30d76da49920a6b': [57264, 57264],
 'a7657cdff2d4dd8f': [69957, 69957],
 '7575bbfbff8d8add': [127355, 127355],
 '215b6e59c204d81a': [52957],
 'acc4be119fe087bf': [58225],
 'ed55e4e8f1b53cd7': [54608],
 'ecad4a8a8f909343': [70142, 70142],
 'b04181fdf29965f9': [58225],
 'ec16a4d0bd73f945': [54608],
 '87032d6a4e7f8892': [69782],
 '5851d5f7e3c9b23': [58225],
 'bcd53a0c5a38f3d7': [54608],
 'd6f0120f3b93186e': [58225],
 '2507fd7485778b8c': [70017],
 '499137b6233829cb': [58225],
 'd3a649c8703dcc26': [54608],
 '72ff9b04e133cf8': [58225],
 '896cbf13f526f024': [58225],
 'f97e